## Toy Differential Privacy - Simple Database Queries

In the context of Deep Learning, Differential Privacy ensures that the DL algorithms learns only what is is supposed to learn from the data while ignoring what it is not supposed to learn from the data. This is called Differential Privacy (DP). But what is DP?





**Differential Privacy (DP):** 
The main goal is to ensure that statistical analysis techniques does not compromise the privacy of any particular individual in the dataset. 

*  Classical privacy defination: privacy is preserved if after the analysis is done, the analyzer does not know anything about the people in the dataset. (No information leak from the data.)

- The Netflix Prize (anonymized simply! Attacked using the Linkage Attack)
- Anonymization is not enough to protect our privacy

A modern definition by Cynthia Dwork:
DP describes a promise made by the data holder to a data subject that:  "you will not be affected by allowing your data to be used in any study or analysis , no matter what other studies, datasets, or information sources are available".


**DP** is a mathematical definition of privacy. In the simplest setting, consider an algorithm that analyzes a dataset and computes statistics about it (such as the data's mean, variance, median, mode, etc.). Such an algorithm is said to be differentially private if by looking at the output, one cannot tell whether any individual's data was included in the original dataset or not. 




---


In this section we're going to play around with Differential Privacy in the context of a database query. The database is going to be a VERY simple database with only one boolean vector. Each row corresponds to a person. Each value corresponds to whether or not that person has a certain private attribute (such as whether they have a certain disease, or whether they are above/below a certain age). 

We are then going to learn how to know whether a database query over such a small database is differentially private or not - and more importantly - what techniques are at our disposal to ensure various levels of privacy


### First We Create a Simple Database

Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [93]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5 # returns a new Boolean tensor with each value > 0.5 set to 1 otherwise set to 0
db

tensor([1, 0, 1,  ..., 0, 1, 1], dtype=torch.uint8)

## Project: Generate Parallel Databases

Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?". Thus, in order to check this, we must construct what we term "parallel databases" which are simply databases with one entry removed. 

In this first project, I want you to create a list of every parallel database to the one currently contained in the "db" variable. Then, I want you to create a function which both:

- creates the initial database (db)
- creates all parallel databases

In [0]:
import torch

In [95]:
def create_db(entries):
  return torch.rand(entries) > 0.5
db = create_db(5000)
db

tensor([0, 0, 0,  ..., 1, 0, 1], dtype=torch.uint8)

In [0]:
# create a function to create paraller db
def get_parallel_dbs(db):

    parallel_dbs = list()

    for i in range(len(db)):
        pdb = torch.cat((db[0:i], db[i+1:]))
        parallel_dbs.append(pdb)
    #print(f'A databse of size {db.shape} was created')
    return parallel_dbs

pdb = get_parallel_dbs(db)


In [0]:
# a helper function for the future
def get_db_and_parallel(num_entries):
  db = torch.rand(num_entries) > 0.5
  pdb = get_parallel_dbs(db)
  return db, pdb

# Lesson: Towards Evaluating The Differential Privacy of a Function

Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. As mentioned previously, this is about evaluating whether the output of a query changes when we remove someone from the database. Specifically, we want to evaluate the *maximum* amount the query changes when someone is removed (maximum over all possible people who could be removed). So, in order to evaluate how much privacy is leaked, we're going to iterate over each person in the database and measure the difference in the output of the query relative to when we query the entire database. 

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [0]:
db, pdbs = get_db_and_parallel(5000)

In [0]:
def query(db):
  return db.sum()

In [100]:
db_result = query(db)
print(db_result)

tensor(2518)


In [0]:


sensitivity = 0  # maximum difference between query(db) and each query(pdb)
for pdb in pdbs:
  sensitivity = max(torch.abs(query(pdb) - db_result), sensitivity)
  
# return sensitivity
  # return the value of sensitivity
  # loop on each pdb in the pdbs and calculate its distance
  # find the maximum distance <-- that's your sensitivity

In [102]:
sensitivity

tensor(1)

# Exercise - Evaluating the Privacy of a Function

In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). This value is called "sensitivity", and it corresponds to the function we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

First, create a function that will accept (1) a query function and (2) a num_items to generate the db and the pdb: 

**Hint:** def sensitivity(query, num_items)

In [0]:
# define a function to calculate the sensitivity of a query
def sensitivity(query, num_items):
  sensitive = 0
  
  # generate the db and the pdbs
  db, pdbs = get_db_and_parallel(num_items)
  
  # query the db
  db_result = query(db)
  print('DB Result: {}'.format(db_result))
  
  # return the value of sensitivity
  # loop on each pdb in the pdbs and calculate its distance
  # find the maximum distance <-- that's your sensitivity
  for pdb in pdbs:
    pdb_query = query(pdb)
    
#     dist = torch.abs(pdb_query - db_result)
    
#     if dist > sensitive:
#       sensitive = dist
    
    sensitive = max(torch.abs(query(pdb) - db_result), sensitive)
    
  return sensitive
  

In [0]:
# recreate the sum query we created earlier
def sum_query(db):
  return db.sum()

In [105]:
# test your sensitivity function with the sum query
print(sensitivity(sum_query, 5000))

DB Result: 2481
tensor(1)


In [0]:
# create a new query function, which finds the mean rather than the sum
def mean_query(db):
  return db.float().mean()

In [107]:
# test your sensitivity function of the Mean query
# how much the output value (query) is using informaiton frmo each of the participating observations
print('Sensitivity: {}'.format(sensitivity(mean_query, 1000)))

DB Result: 0.5230000019073486
Sensitivity: 0.0005235075950622559


Wow! That sensitivity is WAY lower. Note the intuition here. "Sensitivity" is measuring how sensitive the output of the query is to a person being removed from the database. For a simple sum, this is always 1, but for the mean, removing a person is going to change the result of the query by rougly 1 divided by the size of the database (which is much smaller). Thus, "mean" is a VASTLY less "sensitive" function (query) than SUM.

# Exercise: Calculate L1 Sensitivity For Threshold

In this exercise, you need to calculate the sensitivty for the "threshold" function. 

- First compute the sum over the database (i.e. sum(db)) and return whether that sum is greater than a certain threshold.

- Then, create databases of size 10 and threshold of 5 and calculate the sensitivity of the function. 

- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [0]:
# define a function to calculate the sensitivity of a query
def sensitivity(query, num_items, threshold):
  sensitive = 0
  
  # generate the db and the pdbs
  db, pdbs = get_db_and_parallel(num_items)
  
  # query the db
  db_result = query(db, threshold)
    
  # return the value of sensitivity
  # loop on each pdb in the pdbs and calculate its distance
  # find the maximum distance <-- that's your sensitivity
  for pdb in pdbs:
    pdb_query = query(pdb, threshold)
    
    sensitive = max(torch.abs(pdb_query.int() - db_result.int()), sensitive)
    
  return sensitive

In [0]:
# define a fucntion to query the sum and return wethere the sum is greater or less than a threshold
# 
def query(db, threshold):
  return db.sum() > threshold

In [110]:
# the sensitivity with db of size 10 and threshold of 5
print(sensitivity(query, 10, 5))

tensor(0, dtype=torch.int32)


In [111]:
# repeat for 10 times
for i in range(10):
  print(sensitivity(query, 10, 5))

tensor(1, dtype=torch.int32)
tensor(0, dtype=torch.int32)
tensor(1, dtype=torch.int32)
tensor(0, dtype=torch.int32)
tensor(0, dtype=torch.int32)
tensor(1, dtype=torch.int32)
tensor(1, dtype=torch.int32)
tensor(0, dtype=torch.int32)
tensor(0, dtype=torch.int32)
tensor(0, dtype=torch.int32)


# A Basic Differencing Attack

Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

# Exercise:  Perform a Differencing Attack on Row 10

In this project, construct a database and then demonstrate how you can use  different queries (sum, threshold, and mean) to explose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [0]:
# Perform a differencing attack using the sum query on row 10
db = create_db(10)


In [113]:
db

tensor([0, 0, 0, 0, 0, 1, 0, 0, 1, 1], dtype=torch.uint8)

In [0]:
def query(db):
  return db.sum()

In [115]:
def diff_attack(db, idx):
  db_result = query(db)
  
  pdb = torch.cat((db[0:idx], db[idx+1:]))
  pdb_result = query(pdb)
  
  if pdb_result == db_result:
    return 0
  else:
    return 1
  
diff_attack(db, 7)


0

In [0]:
def query(db):
  return db.float().mean()

In [117]:
# Perform a differencing attack using the mean query on row 10
def diff_attack(db, idx):
  db_result = query(db)
  
  pdb = torch.cat((db[0:idx], db[idx+1:]))
  pdb_result = query(pdb)
  
  if pdb_result > db_result:
    return 0
  else:
    return 1
  
diff_attack(db, 9)


1

In [118]:
db = create_db(10)
db

tensor([0, 1, 1, 1, 0, 1, 1, 1, 0, 0], dtype=torch.uint8)

In [0]:
idx = 2
pdb = torch.cat((db[0:idx], db[idx+1:]))

In [120]:
# Perform a differencing attack using the threshold query on row 10
(1 - (sum(pdb) > (sum(db)-1)))


tensor(1, dtype=torch.uint8)

# Project: Local Differential Privacy

As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, the this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results. 

Let's implement this local DP for our database before!

In [0]:
def generate_noisy_db(db):
  # Create first and second flip tensors
  flip_1 = torch.rand(db.shape) > 0.5
  flip_2 = torch.rand(db.shape) > 0.5
  
  # Copy original DB
  noise_db = torch.clone(db)
  
  # flip coins
  noise_db = (noise_db * flip_1) + ((~flip_1) * flip_2)
  
  return noise_db 

def truth_value(query_output):
  return 2 * (query_output - 0.25)

def noise_analysis(num_entries, query):
  db = create_db(num_entries)
  noise_db = generate_noisy_db(db)
  
  og_query = query(db)
  noise_query = query(noise_db)
  
  print('Original query: {}'.format(og_query))
  print('Noise query: {}'.format(noise_query))
  print('Noise Truth Value: {}'.format(truth_value(noise_query)))
  
  return og_query, noise_query

In [0]:
def query(db):
  return db.float().mean()

In [167]:
noise_analysis(10, query)
noise_analysis(1000, query)
noise_analysis(10000, query)
noise_analysis(100000, query)

Original query: 0.800000011920929
Noise query: 0.5
Noise Truth Value: 0.5
Original query: 0.5070000290870667
Noise query: 0.5019999742507935
Noise Truth Value: 0.5039999485015869
Original query: 0.4968999922275543
Noise query: 0.5056999921798706
Noise Truth Value: 0.5113999843597412
Original query: 0.5007100105285645
Noise query: 0.5014399886131287
Noise Truth Value: 0.5028799772262573


(tensor(0.5007), tensor(0.5014))

In [0]:

def truth_value(query_output, noise=0.5):
  return (1/noise) * (query_output - (noise * 0.5))

In [125]:
og_query, noise_query = noise_analysis(100, query)
print(truth_value(noise_query, 0.5))

Original query: 0.4300000071525574
Noise query: 0.41999998688697815
Noise Truth Value: 0.3399999737739563
tensor(0.3400)


In [126]:
noise_analysis(10, query)
noise_analysis(100, query)
noise_analysis(1000000, query)

Original query: 0.4000000059604645
Noise query: 0.4000000059604645
Noise Truth Value: 0.30000001192092896
Original query: 0.5799999833106995
Noise query: 0.6100000143051147
Noise Truth Value: 0.7200000286102295
Original query: 0.4999139904975891
Noise query: 0.4996950030326843
Noise Truth Value: 0.49939000606536865


(tensor(0.4999), tensor(0.4997))

# Project: Varying Amounts of Noise

In this project, I want you to augment the randomized response query (the one we just wrote) to allow for varying amounts of randomness to be added. Specifically, I want you to bias the coin flip to be higher or lower and then run the same experiment. 

Note - this one is a bit tricker than you might expect. You need to both adjust the likelihood of the first coin flip AND the de-skewing at the end (where we create the "augmented_result" variable).

In [0]:
# Copy of above functions but with variable noise
def generate_noisy_db(db, noise):
  # Create first and second flip tensors
  flip_1 = torch.rand(db.shape) > noise
  flip_2 = torch.rand(db.shape) > 0.5
  
  # Copy original DB
  noise_db = torch.clone(db)
  
  # flip coins
  noise_db = (noise_db * flip_1) + ((~flip_1) * flip_2)
  
  return noise_db 

In [0]:
def truth_value(query_output, noise=0.5):
  return (1.0/(1-noise)) * (query_output - (noise * 0.5))

In [0]:
def noise_analysis(query, noise, db=None, num_entries=1000):
  if db is None:
    db = create_db(num_entries)
  
  noise_db = generate_noisy_db(db, noise)
  
  og_query = query(db)
  noise_query = query(noise_db)
  
  print('Noise value: {}'.format(noise))
  print('Original query: {}'.format(og_query))
  print('Noise query: {}'.format(noise_query))
  print('Noise Truth Value: {}'.format(truth_value(noise_query, noise)))
  
  return og_query, noise_query

In [193]:
# New databases created each time
noise_analysis(query, 0.1, num_entries=100000)
noise_analysis(query, 0.5, num_entries=100000)
noise_analysis(query, 0.7, num_entries=100000)
noise_analysis(query, 0.99, num_entries=100000)


Noise value: 0.1
Original query: 0.49900001287460327
Noise query: 0.4993300139904022
Noise Truth Value: 0.49925556778907776
Noise value: 0.5
Original query: 0.4985499978065491
Noise query: 0.4985499978065491
Noise Truth Value: 0.49709999561309814
Noise value: 0.7
Original query: 0.5020700097084045
Noise query: 0.5027300119400024
Noise Truth Value: 0.5091000199317932
Noise value: 0.99
Original query: 0.4984300136566162
Noise query: 0.5000799894332886
Noise Truth Value: 0.5079984664916992


(tensor(0.4984), tensor(0.5001))

In [197]:
# Keep DB constant and vary noise
db = create_db(100000)
noise_analysis(query, 0.0, db=db)
noise_analysis(query, 0.1, db=db)
noise_analysis(query, 0.5, db=db)
noise_analysis(query, 0.7, db=db)
noise_analysis(query, 0.99, db=db)

Noise value: 0.0
Original query: 0.5011699795722961
Noise query: 0.5011699795722961
Noise Truth Value: 0.5011699795722961
Noise value: 0.1
Original query: 0.5011699795722961
Noise query: 0.5020899772644043
Noise Truth Value: 0.5023221969604492
Noise value: 0.5
Original query: 0.5011699795722961
Noise query: 0.49908000230789185
Noise Truth Value: 0.4981600046157837
Noise value: 0.7
Original query: 0.5011699795722961
Noise query: 0.49886998534202576
Noise Truth Value: 0.4962332844734192
Noise value: 0.99
Original query: 0.5011699795722961
Noise query: 0.5017399787902832
Noise Truth Value: 0.6739974021911621


(tensor(0.5012), tensor(0.5017))

# Lesson: The Formal Definition of Differential Privacy

The previous method of adding noise was called "Local Differentail Privacy" because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy. 

However, alternatively we can add noise AFTER data has been aggregated by a function. This kind of noise can allow for similar levels of protection with a lower affect on accuracy. However, participants must be able to trust that no-one looked at their datapoints _before_ the aggregation took place. In some situations this works out well, in others (such as an individual hand-surveying a group of people), this is less realistic.

Nevertheless, global differential privacy is incredibly important because it allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Let's revisit our sum functions.

In [0]:
# noise = laplace(beta)
# epsilon = 0.5
# np.random.laplace(0, beta, 1)

In [132]:
db, pdbs = get_db_and_parallel(100)

def query(db):
    return torch.sum(db.float())

def M(db):
    query(db) + noise

query(db)

tensor(48.)

So the idea here is that we want to add noise to the output of our function. We actually have two different kinds of noise we can add - Laplacian Noise or Gaussian Noise. However, before we do so at this point we need to dive into the formal definition of Differential Privacy.

![alt text](dp_formula.png "Title")

_Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf_

This definition does not _create_ differential privacy, instead it is a measure of how much privacy is afforded by a query M. Specifically, it's a comparison between running the query M on a database (x) and a parallel database (y). As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon, but that occasionally this constraint won't hold with probability delta. Thus, this theorem is called "epsilon delta" differential privacy.

# Epsilon

Let's unpack the intuition of this for a moment. 

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

# Delta

Delta is basically the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than it is for others. For example, you may remember when we were calculating the sensitivity of threshold, most of the time sensitivity was 0 but sometimes it was 1. Thus, we could calculate this as "epsilon zero but non-zero delta" which would say that epsilon is perfect except for some probability of the time when it's arbitrarily higher. Note that this expression doesn't represent the full tradeoff between epsilon and delta.

# Lesson: How To Add Noise for Global Differential Privacy

In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy. In particular, we're going to leave behind the Local Differential privacy previously discussed and instead opt to focus on Global differential privacy. 

So, to sum up, this lesson is about adding noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

### Querying Repeatedly

- if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).

In [0]:
import numpy as np

In [0]:
# define a function to calculate the sensitivity of a query
def sensitivity(query, db, pdbs):
  sensitive = 0
  
  # query the db
  db_result = query(db)
  
  # return the value of sensitivity
  # loop on each pdb in the pdbs and calculate its distance
  # find the maximum distance <-- that's your sensitivity
  for pdb in pdbs:
    pdb_query = query(pdb)
    
    sensitive = max(torch.abs(query(pdb) - db_result), sensitive)
    
  return sensitive

In [0]:
def lap_noise(sensitivity, epsilon):
  return torch.from_numpy(np.random.laplace(0, torch.div(sensitivity, epsilon), 1)).float()

In [0]:
def sum_query(db):
  return torch.sum(db.float())

In [0]:
def M(db, query):
  return torch.add(query(db), noise)

In [0]:
db, pdbs = get_db_and_parallel(50)

In [0]:
eps = 0.5

In [140]:
query(db)

tensor(29.)

In [141]:
sensitive = sensitivity(sum_query, db, pdbs)
noise = lap_noise(sensitive, eps)
print(noise)
query = sum_query

print('Original query: {}'.format(sum_query(db)))
print('Global diff privacy result: {}'.format(M(db, sum_query)))

noise = lap_noise(sensitive, eps)
print('PDB noise: {}'.format(noise))
print('PDB og query: {}'.format(sum_query(pdbs[0])))
print('PDB 0 diff privace result: {}'.format(M(pdbs[0], sum_query)))

tensor([4.1392])
Original query: 29.0
Global diff privacy result: tensor([33.1392])
PDB noise: tensor([10.4797])
PDB og query: 29.0
PDB 0 diff privace result: tensor([39.4797])


# Project: Create a Differentially Private Query

In this project, I want you to take what you learned in the previous lesson and create a query function which sums over the database and adds just the right amount of noise such that it satisfies an epsilon constraint. Write a query for both "sum" and for "mean". Ensure that you use the correct sensitivity measures for both.

In [0]:
import numpy as np

In [0]:
# define a function to calculate the sensitivity of a query
def sensitivity(query, db, pdbs):
  sensitive = 0
  
  # query the db
  db_result = query(db)
  
  # return the value of sensitivity
  # loop on each pdb in the pdbs and calculate its distance
  # find the maximum distance <-- that's your sensitivity
  for pdb in pdbs:
    pdb_query = query(pdb)
    
    sensitive = max(torch.abs(query(pdb) - db_result), sensitive)
    
  return sensitive

In [0]:
def lap_noise(sensitivity, epsilon):
  return torch.from_numpy(np.random.laplace(0, torch.div(sensitivity, epsilon), 1)).float()

In [0]:
def sum_query(db):
  return torch.sum(db.float())

In [0]:
def mean_query(db):
  return torch.mean(db.float())

In [0]:
def M(db, query):
  return torch.add(query(db), noise)

In [0]:
epsilons = [0.1, 0.2, 0.3, 0.5, 0.7, 0.9, 0.99]
queries = [sum_query, mean_query]

In [0]:
db, pdbs = get_db_and_parallel(10000)

In [150]:
for query in queries:
  sense = sensitivity(query, db, pdbs)
  
  for eps in epsilons:
    print(eps)
    noise = lap_noise(sense, eps)
    
    m_db = M(db, query)
    print("Original query: {}".format(query(db)))
    print("Original M: {}".format(m_db))
        

0.1
Original query: 4963.0
Original M: tensor([4979.1167])
0.2
Original query: 4963.0
Original M: tensor([4971.5415])
0.3
Original query: 4963.0
Original M: tensor([4966.9717])
0.5
Original query: 4963.0
Original M: tensor([4964.5933])
0.7
Original query: 4963.0
Original M: tensor([4967.8936])
0.9
Original query: 4963.0
Original M: tensor([4963.4614])
0.99
Original query: 4963.0
Original M: tensor([4963.4531])
0.1
Original query: 0.49630001187324524
Original M: tensor([0.4971])
0.2
Original query: 0.49630001187324524
Original M: tensor([0.4965])
0.3
Original query: 0.49630001187324524
Original M: tensor([0.4963])
0.5
Original query: 0.49630001187324524
Original M: tensor([0.4962])
0.7
Original query: 0.49630001187324524
Original M: tensor([0.4961])
0.9
Original query: 0.49630001187324524
Original M: tensor([0.4963])
0.99
Original query: 0.49630001187324524
Original M: tensor([0.4964])


# Lesson: Differential Privacy for Deep Learning

So in the last lessons you may have been wondering - what does all of this have to do with Deep Learning? Well, these same techniques we were just studying form the core primitives for how Differential Privacy provides guarantees in the context of Deep Learning. 

Previously, we defined perfect privacy as "a query to a database returns the same value even if we remove any person from the database", and used this intuition in the description of epsilon/delta. In the context of deep learning we have a similar standard.

Training a model on a dataset should return the same model even if we remove any person from the dataset.

Thus, we've replaced "querying a database" with "training a model on a dataset". In essence, the training process is a kind of query. However, one should note that this adds two points of complexity which database queries did not have:

    1. do we always know where "people" are referenced in the dataset?
    2. neural models rarely never train to the same output model, even on identical data

The answer to (1) is to treat each training example as a single, separate person. Strictly speaking, this is often overly zealous as some training examples have no relevance to people and others may have multiple/partial (consider an image with multiple people contained within it). Thus, localizing exactly where "people" are referenced, and thus how much your model would change if people were removed, is challenging.

The answer to (2) is also an open problem - but several interesitng proposals have been made. We're going to focus on one of the most popular proposals, PATE.

## An Example Scenario: A Health Neural Network

First we're going to consider a scenario - you work for a hospital and you have a large collection of images about your patients. However, you don't know what's in them. You would like to use these images to develop a neural network which can automatically classify them, however since your images aren't labeled, they aren't sufficient to train a classifier. 

However, being a cunning strategist, you realize that you can reach out to 10 partner hospitals which DO have annotated data. It is your hope to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, you will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

- 1) You'll ask each of the 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels)
- 2) You'll then use each of the 10 partner models to predict on your local dataset, generating 10 labels for each of your datapoints
- 3) Then, for each local data point (now with 10 labels), you will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. We will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint.
- 4) Finally, we will retrain a new model on our local dataset which now has labels. This will be our final "DP" model.

So, let's walk through these steps. I will assume you're already familiar with how to train/predict a deep neural network, so we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [0]:
import numpy as np

In [0]:
num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

In [0]:
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int).transpose(1,0) # fake predictions

In [0]:
new_labels = list()
for an_image in preds:

    label_counts = np.bincount(an_image, minlength=num_labels)

    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [155]:
new_labels

[5,
 1,
 3,
 7,
 1,
 3,
 1,
 2,
 3,
 6,
 3,
 1,
 0,
 4,
 3,
 7,
 1,
 0,
 1,
 8,
 5,
 8,
 3,
 4,
 6,
 5,
 9,
 0,
 1,
 2,
 2,
 1,
 8,
 8,
 3,
 7,
 2,
 9,
 4,
 3,
 9,
 1,
 6,
 1,
 9,
 8,
 3,
 9,
 4,
 9,
 3,
 9,
 9,
 8,
 2,
 3,
 1,
 1,
 7,
 2,
 5,
 6,
 9,
 5,
 3,
 5,
 5,
 5,
 0,
 2,
 8,
 7,
 9,
 8,
 3,
 2,
 9,
 2,
 5,
 2,
 6,
 2,
 4,
 7,
 2,
 2,
 4,
 5,
 6,
 6,
 9,
 7,
 5,
 1,
 3,
 2,
 1,
 0,
 3,
 6,
 8,
 1,
 8,
 0,
 3,
 5,
 2,
 5,
 6,
 5,
 7,
 6,
 3,
 3,
 7,
 1,
 2,
 6,
 8,
 8,
 2,
 3,
 2,
 6,
 4,
 9,
 8,
 9,
 7,
 6,
 0,
 0,
 3,
 0,
 3,
 7,
 4,
 5,
 2,
 2,
 1,
 4,
 8,
 9,
 3,
 8,
 8,
 2,
 0,
 8,
 9,
 5,
 0,
 6,
 0,
 0,
 3,
 8,
 3,
 7,
 6,
 7,
 7,
 6,
 4,
 4,
 8,
 0,
 3,
 0,
 6,
 1,
 5,
 5,
 4,
 4,
 1,
 4,
 2,
 9,
 2,
 9,
 5,
 0,
 7,
 4,
 1,
 1,
 7,
 0,
 9,
 8,
 0,
 3,
 9,
 1,
 1,
 2,
 6,
 1,
 7,
 7,
 2,
 5,
 8,
 5,
 9,
 7,
 8,
 2,
 2,
 0,
 7,
 6,
 4,
 8,
 4,
 2,
 6,
 5,
 2,
 6,
 9,
 6,
 5,
 4,
 1,
 5,
 3,
 5,
 3,
 1,
 2,
 0,
 2,
 8,
 1,
 3,
 7,
 7,
 1,
 2,
 4,
 3,
 9,
 2,
 7,
 0,
 7,
 7,


# PATE Analysis

In [156]:
labels = np.array([9, 9, 3, 6, 9, 9, 9, 9, 8, 2])
counts = np.bincount(labels, minlength=10)
query_result = np.argmax(counts)
query_result

9

In [157]:
from syft.frameworks.torch.differential_privacy import pate

ModuleNotFoundError: ignored

In [0]:
num_teachers, num_examples, num_labels = (100, 100, 10)
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake preds
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers

preds[:,0:10] *= 0

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)

assert data_dep_eps < data_ind_eps



In [0]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

In [0]:
preds[:,0:50] *= 0

In [0]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

# Where to Go From Here


Read:
* Algorithmic Foundations of Differential Privacy: https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf
* Deep Learning with Differential Privacy: https://arxiv.org/pdf/1607.00133.pdf
* The Ethical Algorithm: https://www.amazon.com/Ethical-Algorithm-Science-Socially-Design/dp/0190948205



   
Topics:
* The Exponential Mechanism
* The Moment's Accountant
*  Differentially Private Stochastic Gradient Descent

# Section Project:

For the final project for this section, you're going to train a DP model using this PATE method on the MNIST dataset, provided below.

In [0]:
import torchvision.datasets as datasets
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

In [0]:
train_data = mnist_trainset.train_data
train_targets = mnist_trainset.train_labels

In [0]:
test_data = mnist_trainset.test_data
test_targets = mnist_trainset.test_labels